In [1]:
import pandas as pd

df_train = pd.read_csv('image_labels.csv')
df_train.head()

,filename,label
0,IMG_0345_cropped.jpg,0
1,IMG_0346_cropped.jpg,0
2,IMG_0347_cropped.jpg,0
3,IMG_0348_cropped.jpg,0
4,IMG_0349_cropped.jpg,0


In [2]:
df = pd.read_csv('image_labels.csv')
df = df_train.sample(frac=1, random_state=123) 

train_ratio = 0.7  # 70% for training
val_ratio = 0.15   # 15% for validation
test_ratio = 0.15  # 15% for testing

total_size = df.shape[0]
train_end = round(total_size * train_ratio) 
val_end = round(total_size * (train_ratio + val_ratio))

#create the splits at defined boundaries 
df_train = df.iloc[:train_end] 
df_val = df.iloc[train_end:val_end]
df_test = df.iloc[val_end:]

df_train.to_csv('images_train.csv', index=False)
df_val.to_csv('images_val.csv', index=False)
df_test.to_csv('images_test.csv', index=False)

In [3]:
print(f"Total samples: {total_size}")
print(f"Train: {len(df_train)} ({len(df_train)/total_size:.1%})")
print(f"Val: {len(df_val)} ({len(df_val)/total_size:.1%})")
print(f"Test: {len(df_test)} ({len(df_test)/total_size:.1%})")

Total samples: 43
Train: 30 (69.8%)
Val: 7 (16.3%)
Test: 6 (14.0%)


In [4]:
df_val.head()

,filename,label
0,IMG_0345_cropped.jpg,0
14,IMG_0370_cropped.jpg,0
40,IMG_0595_cropped.jpg,1
9,IMG_0355_cropped.jpg,0
32,IMG_0412_cropped.jpg,0


Creating a Custom Dataset Class

In [5]:
import torch 
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms 
import pandas as pd 

class ImageDataset(Dataset):
    def __init__(self, csv_file, hdf5_file, transform=None): 
        self.data = pd.read_csv(csv_file)
        self.hdf5_file = hdf5_file                           
        self.transform = transform

        with h5py.File(hdf5_file,'r') as f:
            self.dataset_keys = list(f.keys())
            print(f"HDf5 keys: {self.dataset_keys}")

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        filename = self.data.iloc[idx, 0]
        label = torch.tensor(self.data.iloc[idx, 1], dtype=torch.long)
    
        original_idx = self.data.index[idx]
        hdf5_key = f'image_{original_idx}'
        
        with h5py.File(self.hdf5_file, 'r') as f:
            if hdf5_key in f:
                image_data = f[hdf5_key][:]
            else:
                raise ValueError(f"Cannot find HDF5 key '{hdf5_key}' for filename {filename}")
        
        # Convert to PIL Image
        image = Image.fromarray(image_data.astype('uint8'))
        
        if self.transform:
            image = self.transform(image)
        
        return image, label
        


In [42]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std = [0.229, 0.224, 0.225]) # what are the numbers again
])

#now you create the data
train_dataset = ImageDataset('images_train.csv', 'image_dataset.h5',transform = transform)
val_dataset = ImageDataset('images_val.csv', 'image_dataset.h5',transform = transform)
test_dataset = ImageDataset('images_test.csv', 'image_dataset.h5',transform = transform)

# then call the dataloaders
train_loader = DataLoader(train_dataset, batch_size = 4, shuffle= True)
val_loader = DataLoader(val_dataset, batch_size = 4, shuffle= True)
test_loader = DataLoader(test_dataset, batch_size = 4, shuffle= True)


HDf5 keys: ['image_0', 'image_1', 'image_10', 'image_11', 'image_12', 'image_13', 'image_14', 'image_15', 'image_16', 'image_17', 'image_18', 'image_19', 'image_2', 'image_20', 'image_21', 'image_22', 'image_23', 'image_24', 'image_25', 'image_26', 'image_27', 'image_28', 'image_29', 'image_3', 'image_30', 'image_31', 'image_32', 'image_33', 'image_34', 'image_35', 'image_36', 'image_37', 'image_38', 'image_39', 'image_4', 'image_40', 'image_41', 'image_42', 'image_43', 'image_44', 'image_45', 'image_46', 'image_47', 'image_48', 'image_49', 'image_5', 'image_50', 'image_51', 'image_52', 'image_53', 'image_54', 'image_55', 'image_56', 'image_57', 'image_58', 'image_59', 'image_6', 'image_60', 'image_61', 'image_62', 'image_63', 'image_64', 'image_65', 'image_66', 'image_67', 'image_68', 'image_7', 'image_8', 'image_9']
HDf5 keys: ['image_0', 'image_1', 'image_10', 'image_11', 'image_12', 'image_13', 'image_14', 'image_15', 'image_16', 'image_17', 'image_18', 'image_19', 'image_2', 'imag

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleImageClassifier(nn.Module):
    def __init__(self):
        super(SimpleImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1) #tiny CNN
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # Calculate the size after convolutions
        # For 224x224 input: after 2 pooling layers = 56x56 but hmm what does this do
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)  #Binary classification as high or low
        self.dropout = nn.Dropout(0.2) #to help out with learning best features 
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [44]:
# Initialize model, loss, optimizer
model = SimpleImageClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

SimpleImageClassifier(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [45]:
def train_one_epoch(modek, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels, = images.to(device), labels.to(device)

        optimizer.zero_grad() #initialize gradient, don't carry from previous 
        outputs = model(images) #outputs are values from the model 
        loss = criterion(outputs, labels) #compare values
        loss.backward() #back prop to update weights
        optimizer.step() #update weights with the optimizer!!

        running_loss += loss.item() 
        _, predicted = torch.max(outputs.data, 1) #get the predictions from model 
        total += labels.size(0) #get the toal 
        correct += (predicted == labels).sum().item() #add to the correct

    return running_loss /len(train_loader), 100 * correct / total #calculate accuracy


In [ ]:
def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels, = images.to(device), labels.to(device)
            outputs = model(images) #no optimization here 
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return val_loss / len+(val_loader), 100 * correct / total 

In [47]:
num_epochs = 5

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(epoch+1, num_epochs)
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print('-' * 50)

1 5
  Train Loss: 1.8111, Train Acc: 63.33%
  Val Loss: 1.1182, Val Acc: 14.29%
--------------------------------------------------
2 5
  Train Loss: 0.7853, Train Acc: 53.33%
  Val Loss: 0.4805, Val Acc: 85.71%
--------------------------------------------------
3 5
  Train Loss: 0.4149, Train Acc: 76.67%
  Val Loss: 0.4963, Val Acc: 85.71%
--------------------------------------------------
4 5
  Train Loss: 0.3554, Train Acc: 76.67%
  Val Loss: 0.5367, Val Acc: 85.71%
--------------------------------------------------
5 5
  Train Loss: 0.2727, Train Acc: 90.00%
  Val Loss: 0.6317, Val Acc: 85.71%
--------------------------------------------------
